In [1]:
#!pip3 install "gym[atari]"
import gym
import sys
from PIL import Image, ImageDraw
import numpy as np
import pandas as pd
import neural_tape_controller
#Положительные числа - положительные награды.
import tasks 
import optimize
import  pickle
import torch

In [2]:
def query_environment(name):
    env = gym.make(name)
    spec = gym.spec(name)
    print(f"Action Space: {env.action_space}")
    print(f"Observation Space: {env.observation_space}")
    print(f"Max Episode Steps: {spec.max_episode_steps}")
    print(f"Nondeterministic: {spec.nondeterministic}")
    print(f"Reward Range: {env.reward_range}")
    print(f"Reward Threshold: {spec.reward_threshold}")
query_environment("ALE/Tetris-v5")

Action Space: Discrete(5)
Observation Space: Box(0, 255, (210, 160, 3), uint8)
Max Episode Steps: 27000
Nondeterministic: False
Reward Range: (-inf, inf)
Reward Threshold: None


A.L.E: Arcade Learning Environment (version 0.7.5+db37282)
[Powered by Stella]


In [3]:
def encode_simple(X):
  #как сделать простое кодирование? Ну давай разобьём картинку двумя разными сетками и измерим яркость
  #на входе одиночная картинка
  y_lst = []
  sz = np.shape(X)
  count_squares = 12
  for i in range(count_squares):
    for j in range(count_squares):
      x = X[int(i*sz[0]/count_squares):int((i+1)*sz[0]/count_squares), int(j*sz[1]/count_squares):int((j+1)*sz[1]/count_squares)]
      y_lst.append(np.nanmean(x))

  count_squares = 9
  for i in range(count_squares):
    for j in range(count_squares):
      x = X[int(i*sz[0]/count_squares):int((i+1)*sz[0]/count_squares), int(j*sz[1]/count_squares):int((j+1)*sz[1]/count_squares)]
      y_lst.append(np.nanmean(x))

  count_squares = 5
  for i in range(count_squares):
    for j in range(count_squares):
      x = X[int(i*sz[0]/count_squares):int((i+1)*sz[0]/count_squares), int(j*sz[1]/count_squares):int((j+1)*sz[1]/count_squares)]
      y_lst.append(np.nanmean(x))

  #count_squares = 2
  #for i in range(count_squares):
  #  for j in range(count_squares):
  #    x = X[int(i*sz[0]/count_squares):int((i+1)*sz[0]/count_squares), int(j*sz[1]/count_squares):int((j+1)*sz[1]/count_squares)]
  #    y_lst.append(np.nanmean(x))
  #размер: 62
  return np.array(y_lst)

In [4]:
n_actions = 5
embed_size = 250#это эмбеддинг картинки
nt = neural_tape_controller.nt_controller(input_size=embed_size,output_size=n_actions)
try:
    with open('./genoms/best_genom_tetris.pkl', 'rb') as f:
        genom = pickle.load(f)
        genom = genom[-1]
    print('loaded successfully')
except Exception:
    genom = nt.nn.disassemble_genom()

loaded successfully


In [5]:
#3, 1, 4, 4

In [6]:
n_actions = 5
embed_size = 100800#это эмбеддинг картинки
nt = neural_tape_controller.nt_controller(tacts=1,input_size=embed_size,output_size=n_actions)
genom = nt.nn.disassemble_genom()

In [7]:
def test_atary(genom, draw=False,game_name="ALE/Tetris-v5",seed=0,controller=None):

  env = gym.make(game_name)
  env.reset()
  n_actions = env.action_space.n
  state_dim = env.observation_space.shape#здесь картинка
  #print('n_actions',n_actions,'state_dim',state_dim)
  #1/0
  state_dim = embed_size#это эмбеддинг картинки
  if controller==None: controller = neural_tape_controller.nt_controller(tacts=1,genom=np.array(genom),input_size=state_dim,output_size=n_actions) 
  out_tape = np.zeros(30)
  reward_sum = 0

  #seed=1
  #np.random.seed(seed)
  #env.seed(seed)
  while True:
  #for i in range(200):
      action = np.ravel(out_tape)
      action += np.random.rand(len(action))*np.std(action)*0.05#игра детерминистическая, иначе рандома не будет
      #print('action',int(np.argmax(action)),action)
      #исполнить env
      state, reward, done,_ = env.step(int(np.argmax(action)))
        
      shp = np.shape(state)
      #print(shp)
      #1/0
      state = np.reshape(state,[shp[2],shp[0],shp[1]])
      #print(np.shape(state))
        
      t=pd.Timestamp.now()
      if done:
        break
      #state = encode_mobnet(np.array([state]))
      #state = encode_simple(state)
      reward_sum += (reward + 0.1)
      out_tape = controller.act(torch.tensor(state),reward,done)
      if draw:
        globals()['video'].append(Image.fromarray(env.render("rgb_array")))
  return reward_sum, controller

In [8]:
def multy_test(genom,draw=False):
  if draw:
    globals()['video'] = []

  game_name = "ALE/Tetris-v5"
  q_arr = []
  controller = neural_tape_controller.nt_controller(tacts=1,genom=np.array(genom),input_size=100800,output_size=n_actions) 
  for i in range(3):
    q,controller = test_atary(genom, draw=draw,game_name=game_name,controller=controller)
    q_arr.append(q*(i+1))
  return np.sum(q_arr)/len(q_arr)

In [9]:
#(210, 160, 3)

In [10]:
%%time
q,_=test_atary(genom,game_name="ALE/Tetris-v5", draw=False)
print('q=',q)

q= 56.50000000000053
CPU times: user 1min 27s, sys: 48.9 ms, total: 1min 27s
Wall time: 15 s


In [11]:
%%time
q=multy_test(genom)
print('q=',q)
#q=multy_test(genom)
#print('q=',q)

q= 121.93333333333449
CPU times: user 5min 50s, sys: 340 ms, total: 5min 50s
Wall time: 1min


In [12]:
%%time
q=test_atary(genom)
print('q=',q)

q= (69.70000000000032, <neural_tape_controller.nt_controller object at 0x7f2ddccad9a0>)
CPU times: user 3min 53s, sys: 269 ms, total: 3min 53s
Wall time: 41.1 s


In [13]:
print(pd.Timestamp.now())

2022-06-07 19:50:32.940569


In [12]:
print(1)
opt = optimize.optimizer(multy_test, genom_size=len(genom),parallel_cores=1,init_file='genoms/genom1.pkl',history_file='history/history2.pkl')

1
history loaded successfully


In [15]:
try:
    with open('genoms/best_genom_tetris.pkl', 'rb') as f:
        opt.best_genoms = pickle.load(f)
    print('loaded successfully')
except Exception:
    pass

loaded successfully


In [13]:

opt.function(opt.best_genoms[-1])

for i in range(1000):
    print('opt#',i)
    opt.optimize()
    with open('genoms/best_genom_tetris.pkl', 'wb') as f:
        pickle.dump(opt.best_genoms,f,protocol=pickle.HIGHEST_PROTOCOL)
        print('WRITTEN')

opt# 0
scores for optimizers augmented [-8.00450000e+01  3.26644167e+02  1.01664167e+03  1.65200000e+00
  2.22666667e+00 -4.20000000e-02 -7.50000000e-02  6.66666667e-02
  1.00000000e+10]
chosen evol_narrow previous_result: nan per tacts: nan
iteration 0 y= [115.  111.8]
iteration 1 y= [120.63333333 117.1       ]
iteration 2 y= [128.9        122.86666667]
iteration 3 y= [126.5 125. ]
iteration 4 y= [130.76666667 118.4       ]
iteration 5 y= [117.56666667 117.4       ]
iteration 6 y= [123.6        123.03333333]
iteration 7 y= [126.2        120.43333333]
iteration 8 y= [126.83333333 123.1       ]
iteration 9 y= [123.2 119.9]
iteration 10 y= [127.93333333 123.9       ]
iteration 11 y= [132.         118.46666667]
iteration 12 y= [122.06666667 120.26666667]
iteration 13 y= [136.43333333 115.53333333]
iteration 14 y= [133.46666667 133.13333333]
iteration 15 y= [136.76666667 119.63333333]
iteration final y= [138.6        135.46666667]
result evol_narrow previous_gain: 23.59999999999893 per tac

Traceback (most recent call last):
  File "/home/nikita/.local/lib/python3/site-packages/IPython/core/interactiveshell.py", line 3397, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/.private/nikita/ipykernel_4653/1451368021.py", line 5, in <cell line: 3>
    opt.optimize()
  File "/home/nikita/neuroevolution-toolbox/optimize.py", line 88, in optimize
    elif chosen_optimizer=='gradient_wide_50':
  File "/home/nikita/neuroevolution-toolbox/optimize.py", line 331, in rel_coord_default
  File "/home/nikita/neuroevolution-toolbox/optimize.py", line 358, in relative_coordinate_descent
    genom_local[idx:idx+stripe] *= 1+step
  File "/tmp/.private/nikita/ipykernel_4653/4178496395.py", line 9, in multy_test
    q,controller = test_atary(genom, draw=draw,game_name=game_name,controller=controller)
  File "/tmp/.private/nikita/ipykernel_4653/3154230333.py", line 37, in test_atary
    out_tape = controller.act(state,reward,done)
  File "/home/nikita/neuroevolutio

In [20]:
with open('./genoms/genom1.pkl', 'rb') as f:
        genom = pickle.load(f)
        #genom = genom[-1]

In [21]:
q=multy_test(genom,draw=True)
print('q=',q)

q= 90.86666666666578


In [22]:
video[0].save(
    './out_videos/tetris.gif',
    save_all=True,
    append_images=video[1:], 
    optimize=True,
    duration=100,
    loop=0
)

In [ ]:
#noise stategy
#with open('genoms/best_genom_tetris.pkl', 'rb') as f:
    #genom = pickle.load(f)[-1]
globals()['video'] = []
noise = np.random.normal(size=[len(genom)])*1
for i in range(1):
    genom = genom+noise
    q=test_atary(genom, draw=True,game_name="ALE/Tetris-v5",seed=0)
    np.random.seed(i)
    noise = np.random.normal(size=[len(genom)])*1



In [ ]:
video[0].save(
    './out_videos/tetris.gif',
    save_all=True,
    append_images=video[1:], 
    optimize=True,
    duration=10,
    loop=0
)

In [ ]:
with open('genoms/best_genom_tetris.pkl', 'wb') as f:
    pickle.dump(genom,f,protocol=pickle.HIGHEST_PROTOCOL)
    print('WRITTEN')